# **LLM + Function Call with Semantic Kernel Memory**
This will guide you through **Large Language Models (LLMs)**, and **Semantic Kernel Plugin** in Python.

In [ ]:
%pip install semantic-kernel >nul 2>&1
% pip install ollama >nul 2>&1


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%` not found.


## **Exercise 1: Running a Basic Prompt**
### **What You Will Learn**
- How to send a basic query and receive a response

In [20]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.ollama import OllamaChatCompletion
from semantic_kernel.connectors.ai.ollama.ollama_prompt_execution_settings import OllamaChatPromptExecutionSettings
from semantic_kernel.functions import KernelArguments

# Initialize the kernel
kernel = Kernel()

# Configure the Ollama chat completion service
model_name = "llama3.2"  # Ensure this model is pulled and available
ollama_endpoint = "http://localhost:11434"
chat_completion_service = OllamaChatCompletion(ai_model_id=model_name, host=ollama_endpoint)

# Create request settings for Ollama
request_settings = OllamaChatPromptExecutionSettings()

# Register the Ollama service with the kernel
kernel.add_service(chat_completion_service)

# User query
user_input = "What is the capital of Israel?"



# Invoke the chat function
result = await kernel.invoke_prompt(user_input)

# Process the result
if result:
    response = result.value[0]
    print(f"Chatbot:> {response}")


Chatbot:> The capital of Israel is Jerusalem.


## **Exercise 2: Using Time Plugin to Provide Time Information**

In this exercise, you will:
- Utilize the TimePlugin to retrieve and display the current time.
- Integrate the plugin with the Semantic Kernel to handle time-related queries.
- Create a function that responds to user input asking for the current time.
- Test your implementation by sending queries like "What time is it now?" and verifying that the chatbot returns the correct time.


In [6]:
from semantic_kernel import Kernel
from semantic_kernel.core_plugins.math_plugin import MathPlugin
from semantic_kernel.core_plugins.time_plugin import TimePlugin
from semantic_kernel.connectors.ai.ollama import OllamaChatCompletion
from semantic_kernel.connectors.ai.ollama.ollama_prompt_execution_settings import OllamaChatPromptExecutionSettings
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import KernelArguments

# Initialize the kernel
kernel = Kernel()

# Configure the Ollama chat completion service
model_name = "llama3.2"  # Ensure this model is pulled and available
ollama_endpoint = "http://localhost:11434"
chat_completion_service = OllamaChatCompletion(ai_model_id=model_name, host=ollama_endpoint)

# Create request settings for Ollama
request_settings = OllamaChatPromptExecutionSettings()
request_settings = OllamaChatPromptExecutionSettings()
request_settings.function_choice_behavior = FunctionChoiceBehavior.Auto(filters={"excluded_plugins": ["ChatBot"]})

# Register the Ollama service with the kernel
kernel.add_service(chat_completion_service)
kernel.add_plugin(TimePlugin(), plugin_name="time")

# User query
user_input = "What time is it now?"

# Initialize chat history
history = ChatHistory()
history.add_user_message(user_input)

# Update arguments with user input and chat history
arguments = KernelArguments(settings=request_settings)
arguments["user_input"] = user_input
arguments["chat_history"] = history

chat_function = kernel.add_function(
    prompt="{{$chat_history}}{{$user_input}}",
    plugin_name="ChatBot",
    function_name="Chat")
    
# Invoke the chat function
result = await kernel.invoke(chat_function, arguments=arguments)

# Process the result
if result:
    response = result.value[0]
    print(f"Chatbot:> {response}")


Chatbot:> The current time is Sunday, February 23, 2025 10:21 PM.


## **Exercise 3: Using Semantic Kernel Functions to provide information**
Since Semantic Kernel can call a function using instruct models, we are going to use a local OLlama server

### **What You Will Learn**
- Running and playing with a local Ollama server
- Loading a model and chat with it
- Write a code that:
    - Add a plugin and a function that allow the Kernel to get external information


### Using Ollama local server with docker

```
docker run -d --name ollama -p 11434:11434 ollama/ollama:latest
```
#### Start the server

```
docker exec -it ollama ollama serve
```

#### Chat with a model

```
docker exec -it ollama ollama run llama3.2

```

### Using Ollama local server without docker

```
ollama serve
```


In [8]:
import asyncio
import os
from semantic_kernel import Kernel
from semantic_kernel.core_plugins.math_plugin import MathPlugin
from semantic_kernel.core_plugins.time_plugin import TimePlugin
from semantic_kernel.connectors.ai.ollama import OllamaChatCompletion
from semantic_kernel.connectors.ai.ollama.ollama_prompt_execution_settings import OllamaChatPromptExecutionSettings
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import KernelArguments

# Initialize the kernel
kernel = Kernel()


class ProcInfoPlugin:
    @kernel_function(
        name="read_proc_info",
        description="Provides information about the operating system, including load average, memory info, network devices, and uptime."
    )
    async def read_proc_info(self) -> str:
        """Reads system information from /proc and returns a formatted summary."""

        async def read_file(path: str) -> str:
            """Asynchronously reads a file, handling errors gracefully."""
            try:
                return await asyncio.to_thread(lambda: open(path, "r").read().strip())
            except FileNotFoundError:
                return f"Error: {path} not found."
            except PermissionError:
                return f"Error: Permission denied for {path}."
            except Exception as e:
                return f"Error reading {path}: {e}"

        #For Windows WSL - Check the your specific path
        BASE_PATH =  r"\\wsl.localhost\Ubuntu-20.04\proc"

        #For Linux
        #BASE_PATH = "/proc"

        # Read system information files asynchronously
        loadavg, meminfo, netdev, uptime = await asyncio.gather(
            read_file(os.path.join(BASE_PATH, "loadavg")),
            read_file(os.path.join(BASE_PATH, "meminfo")),
            read_file(os.path.join(BASE_PATH, "net/dev")),
            read_file(os.path.join(BASE_PATH, "uptime")),
        )

        # Format output for readability
        formatted_meminfo = "\n".join(meminfo.splitlines()[:10])  # Show first 10 lines
        formatted_netdev = "\n".join(netdev.splitlines()[:5])  # Show first 5 lines

        return (
            f"=== System Information ===\n\n"
            f"📊 **Load Average:**\n{loadavg}\n\n"
            f"🛑 **Memory Info (First 10 Lines):**\n{formatted_meminfo}\n\n"
            f"🌐 **Network Devices (First 5 Lines):**\n{formatted_netdev}\n\n"
            f"⏳ **Uptime:** {uptime} seconds"
        )

kernel.add_plugin(ProcInfoPlugin(), "proc_info_plugin")

# Configure the Ollama chat completion service
model_name = "llama3.2"  # Ensure this model is pulled and available
ollama_endpoint = "http://localhost:11434"
chat_completion_service = OllamaChatCompletion(ai_model_id=model_name, host=ollama_endpoint)

# Create request settings for Ollama
request_settings = OllamaChatPromptExecutionSettings()
request_settings.function_choice_behavior = FunctionChoiceBehavior.Auto(filters={"excluded_plugins": ["ChatBot"]})

# Register the Ollama service with the kernel
kernel.add_service(chat_completion_service)

# User query
user_input = "Provide a summery information about the operating system information"

# Initialize chat history
history = ChatHistory()
history.add_user_message(user_input)

# Update arguments with user input and chat history
arguments = KernelArguments(settings=request_settings)
arguments["user_input"] = user_input
arguments["chat_history"] = history

chat_function = kernel.add_function(
    prompt="{{$chat_history}}{{$user_input}}",
    plugin_name="ChatBot",
    function_name="Chat")
    
# Invoke the chat function
result = await kernel.invoke(chat_function, arguments=arguments)

# Process the result
if result:
    response = result.value[0]
    print(f"Chatbot:> {response}")


Chatbot:> Here's a summary of the operating system information:

**Load Average:** The system is currently running with an average load of 2.10, indicating moderate usage.

**Memory Info:**

* Total memory: 32.8 GB
* Free memory: 25.6 GB
* Available memory: 28.1 GB
* Buffers: 2.8 MB
* Caches: 2.3 GB

**Network Devices:**

* The system is currently unable to read information about the network device "\\wsl.localhost\\Ubuntu-20.04\\proc\\net\\dev" due to an invalid argument.

**Uptime:** The system has been running for approximately 952 minutes (or about 15 hours and 52 minutes) or 41580 seconds, with a second value indicating an extended uptime period of 32 hours and 3 minutes.


## **Exercise 4: Using Semantic Kernel Functions to take action**

### **What You Will Learn**
- Write a code that create a file using prompt instructions
    


In [13]:
import asyncio
import os
from semantic_kernel import Kernel
from semantic_kernel.core_plugins.math_plugin import MathPlugin
from semantic_kernel.core_plugins.time_plugin import TimePlugin
from semantic_kernel.connectors.ai.ollama import OllamaChatCompletion
from semantic_kernel.connectors.ai.ollama.ollama_prompt_execution_settings import OllamaChatPromptExecutionSettings
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import KernelArguments

# Initialize the kernel
kernel = Kernel()


class ProcInfoPlugin:
    @kernel_function(
        name="read_proc_info",
        description="Provides information about the operating system, including load average, memory info, network devices, and uptime."
    )
    async def read_proc_info(self) -> str:
        """Reads system information from /proc and returns a formatted summary."""

        async def read_file(path: str) -> str:
            """Asynchronously reads a file, handling errors gracefully."""
            try:
                return await asyncio.to_thread(lambda: open(path, "r").read().strip())
            except FileNotFoundError:
                return f"Error: {path} not found."
            except PermissionError:
                return f"Error: Permission denied for {path}."
            except Exception as e:
                return f"Error reading {path}: {e}"

        #For Windows WSL - Check the your specific path
        BASE_PATH =  r"\\wsl.localhost\Ubuntu-20.04\proc"

        #For Linux
        #BASE_PATH = "/proc"

        # Read system information files asynchronously
        loadavg, meminfo, netdev, uptime = await asyncio.gather(
            read_file(os.path.join(BASE_PATH, "loadavg")),
            read_file(os.path.join(BASE_PATH, "meminfo")),
            read_file(os.path.join(BASE_PATH, "net/dev")),
            read_file(os.path.join(BASE_PATH, "uptime")),
        )

        # Format output for readability
        formatted_meminfo = "\n".join(meminfo.splitlines()[:10])  # Show first 10 lines
        formatted_netdev = "\n".join(netdev.splitlines()[:5])  # Show first 5 lines

        return (
            f"=== System Information ===\n\n"
            f"📊 **Load Average:**\n{loadavg}\n\n"
            f"🛑 **Memory Info (First 10 Lines):**\n{formatted_meminfo}\n\n"
            f"🌐 **Network Devices (First 5 Lines):**\n{formatted_netdev}\n\n"
            f"⏳ **Uptime:** {uptime} seconds"
        )

kernel.add_plugin(ProcInfoPlugin(), "proc_info_plugin")

class SaveTextFilePlugin:
    @kernel_function(
        name="save_text_file",
        description="When asked to write information to a file, use this function with the specified filename."
    )
    async def save_text_file(self, filename: str, text: str) -> str:
        """Asynchronously writes text content to a file."""
        def write_file():
            with open(filename, "w", encoding="utf-8") as f:
                f.write(text)
        await asyncio.to_thread(write_file)
        return f"File '{filename}' saved successfully."

kernel.add_plugin(SaveTextFilePlugin(), "save_text_file_plugin")

# Configure the Ollama chat completion service
model_name = "llama3.2"  # Ensure this model is pulled and available
ollama_endpoint = "http://localhost:11434"
chat_completion_service = OllamaChatCompletion(ai_model_id=model_name, host=ollama_endpoint)

# Create request settings for Ollama
request_settings = OllamaChatPromptExecutionSettings()
request_settings.function_choice_behavior = FunctionChoiceBehavior.Auto(filters={"excluded_plugins": ["ChatBot"]})

# Register the Ollama service with the kernel
kernel.add_service(chat_completion_service)

# User query
#user_input = "Write a summery information to a file about the operating system information to the file C:\\temp\\system_info.txt"
user_input = "Get the operating system information and Write it to the file C:\\temp\\system_info.txt"
# Initialize chat history
history = ChatHistory()
history.add_user_message(user_input)

# Update arguments with user input and chat history
arguments = KernelArguments(settings=request_settings)
arguments["user_input"] = user_input
arguments["chat_history"] = history

chat_function = kernel.add_function(
    prompt="{{$chat_history}}{{$user_input}}",
    plugin_name="ChatBot",
    function_name="Chat")
    
# Invoke the chat function
result = await kernel.invoke(chat_function, arguments=arguments)

# Process the result
if result:
    response = result.value[0]
    print(f"Chatbot:> {response}")


Chatbot:> Here is the system information written to the file C:\temp\system_info.txt:

C:\temp\System Info (System Information).txt

Load Average:
8.27 5.64 4.13 1/823 3954


Memory Info:
MemTotal:       32811156 kB
MemFree:        25958100 kB
MemAvailable:   27986108 kB
Buffers:            3104 kB
Cached:          2382792 kB
SwapCached:            0 kB
Active:          2005060 kB
Inactive:        4136884 kB
Active(anon):      16104 kB
Inactive(anon):  3772284 kB


Network Devices:
Error reading \\wsl.localhost\Ubuntu-20.04\proc\net/dev: [Errno 22] Invalid argument: '\\\\wsl.localhost\\Ubuntu-20.04\\proc\\net/dev'


Uptime:
95910.41 seconds
4184314.81 seconds

The system information has been written to the file C:\temp\System Info (System Information).txt


## **Final Exercise**

**Exercise 5: Building a Chat with your system Chatbot**

### **1. Persistent Chatbot with History**
- Modify the chatbot so it maintains a conversation history, allowing the user to ask follow-up questions.
- Use **`ChatHistory`** from `semantic_kernel.contents` to store previous messages.

### **2. Modular System Information Functions**
Break down the **`read_proc_info`** function into multiple functions, each dedicated to specific system details:
- **CPU Info:** Read `/proc/cpuinfo`
- **Memory Info:** Read `/proc/meminfo`
- **Disk Usage:** Use `df -h`
- **Running Processes:** Read `/proc/[PID]/status`
- **Network Info:** Parse `/proc/net/dev`

### **3. Extended System Information**
Extend the system information retrieval with:
- **GPU Information:** Use `lspci`
- **Mounted Drives:** Use `mount -v`
- **Kernel Version & OS Info:** Use `uname -a` (Linux)

### **4. Running Whitelisted Processes**
- Introduce a **whitelist** of allowed processes.
- Create a function `run_whitelisted_process(command: str)` that checks if the command is in the whitelist before execution.
- Use `subprocess.run([...])` in Python, ensuring that user input is sanitized to avoid security risks.

### **5. Killing Processes Started by the Chatbot**
- Track processes that the chatbot starts.
- Provide a function to terminate only those processes.
- Use:
  - `os.kill(pid, signal.SIGTERM)`

---

### **Implementation Plan**
- Modify the chatbot to maintain history.
- Refactor system information retrieval into multiple functions.
- Implement process execution control using a **whitelist**.
- Add a function to **terminate chatbot-created processes** safely.




In [ ]:
import asyncio
import os
import psutil
import subprocess
from semantic_kernel import Kernel
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.connectors.ai.ollama import OllamaChatCompletion
from semantic_kernel.connectors.ai.ollama.ollama_prompt_execution_settings import OllamaChatPromptExecutionSettings
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.functions import KernelArguments

# Initialize kernel and chatbot history
kernel = Kernel()
chat_history = ChatHistory()

# --- SYSTEM INFORMATION MODULE ---
class SystemInfoPlugin:
    @kernel_function(name="get_cpu_info", description="Retrieve CPU information.")
    async def get_cpu_info(self) -> str:
        return subprocess.run("lscpu", capture_output=True, text=True).stdout if os.name != "nt" else subprocess.run("wmic cpu get Name", capture_output=True, text=True).stdout

    @kernel_function(name="get_memory_info", description="Retrieve memory usage details.")
    async def get_memory_info(self) -> str:
        return subprocess.run("free -h", capture_output=True, text=True).stdout if os.name != "nt" else subprocess.run("wmic OS get FreePhysicalMemory,TotalVisibleMemorySize", capture_output=True, text=True).stdout
    
    @kernel_function(name="get_disk_info", description="Retrieve disk usage details.")
    async def get_disk_info(self) -> str:
        return subprocess.run("df -h", capture_output=True, text=True).stdout if os.name != "nt" else subprocess.run("wmic logicaldisk get size,freespace,caption", capture_output=True, text=True).stdout
    
    @kernel_function(name="get_network_info", description="Retrieve network interfaces and details.")
    async def get_network_info(self) -> str:
        return subprocess.run("ip a", capture_output=True, text=True).stdout if os.name != "nt" else subprocess.run("ipconfig /all", capture_output=True, text=True).stdout

kernel.add_plugin(SystemInfoPlugin(), "system_info_plugin")

# --- PROCESS MANAGEMENT MODULE ---
class ProcessManagerPlugin:
    allowed_processes = {"ping", "ls", "dir", "echo", "whoami"}
    running_processes = {}
    
    @kernel_function(name="run_whitelisted_process", description="Run a whitelisted system process and return the output.")
    async def run_whitelisted_process(self, command: str) -> str:
        command_name = command.split()[0]
        if command_name not in self.allowed_processes:
            return f"Error: The command '{command_name}' is not allowed."
        
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        self.running_processes[process.pid] = process
        stdout, stderr = process.communicate()
        output = stdout if stdout else stderr
        return f"Output of '{command}':\n{output.strip()}"
    
    @kernel_function(name="kill_process", description="Terminate a process started by the chatbot.")
    async def kill_process(self, pid: int) -> str:
        if pid in self.running_processes:
            self.running_processes[pid].terminate()
            del self.running_processes[pid]
            return f"Process {pid} terminated successfully."
        return "Error: Process ID not found or not started by chatbot."

kernel.add_plugin(ProcessManagerPlugin(), "process_manager_plugin")

# --- CHATBOT SETUP ---
ollama_endpoint = "http://localhost:11434"
model_name = "llama3.2"
chat_service = OllamaChatCompletion(ai_model_id=model_name, host=ollama_endpoint)
kernel.add_service(chat_service)

# Chat settings
request_settings = OllamaChatPromptExecutionSettings()
request_settings.function_choice_behavior = FunctionChoiceBehavior.Auto(filters={"excluded_plugins": ["ChatBot"]})

# Define and register ChatBot plugin implementation
class ChatBotPlugin:
    @kernel_function(name="Chat", description="Chat function that delegates to the Ollama chat service")
    async def Chat(self, chat_history, user_input) -> str:
        result = await kernel.invoke_prompt(user_input)
        return result.value[0] if result and result.value else "No response received."

kernel.add_plugin(ChatBotPlugin(), "ChatBot")

chat_function = kernel.add_function(prompt="{{$chat_history}}{{$user_input}}", plugin_name="ChatBot", function_name="Chat")


while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    
    chat_history.add_user_message(user_input)
    arguments = KernelArguments(settings=request_settings)
    arguments["user_input"] = user_input
    arguments["chat_history"] = chat_history

    result = await kernel.invoke(chat_function, arguments=arguments)
    if result:
        response = result.value[0]
        print(f"Chatbot: {response}")
        #chat_history.add_assistant_message(response)



Chatbot: Based on the output, your IP address is:

* IPv4 Address: 192.168.0.50
* Subnet Mask: 255.255.255.0
* Default Gateway: fe80::ea9c:25ff:fe89:7f08%

Your CPU information is:

* Manufacturer: Intel
* Model Name: Intel(R) Xeon(R) CPU E5-2699 v4 @ 2.20GHz


## **Setting up offline environment**

To set up your environment for running the provided Jupyter Notebook in a disconnected setting, follow these steps:

1. **Download and Prepare Dependencies**: Use the following script to download all necessary models and Docker images. This script should be executed in an environment with internet access.

   ```bash
   #!/bin/bash

   # Create a directory to store all resources
   mkdir -p llm_resources
   cd llm_resources

   # Download the LLaMA 2 model
   git lfs install
   git clone https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF
   mv Llama-2-7B-Chat-GGUF models

   # Pull the Ollama Docker image
   docker pull ollama/ollama:latest
   docker save ollama/ollama:latest -o ollama_latest.tar

   # Create a requirements file for Python dependencies
   cat <<EOF > requirements.txt
   torch
   torchvision
   torchaudio
   faiss-cpu
   numpy
   sentence-transformers
   semantic-kernel
   huggingface_hub
   llama-cpp-python
   EOF

   # Download Python packages
   pip download -r requirements.txt -d python_packages

   echo "All resources have been downloaded and saved in the 'llm_resources' directory."
   ```


   **Instructions**:

   - Run the above script on a machine with internet access.
   - Transfer the `llm_resources` directory to your target offline environment.

2. **Set Up in the Disconnected Environment**:

   - **Install Docker**: Ensure Docker is installed on your offline machine. If not, download the Docker installation package appropriate for your system and transfer it to the machine for installation.

   - **Load the Ollama Docker Image**: Navigate to the `llm_resources` directory and load the Docker image:

     ```bash
     docker load -i ollama_latest.tar
     ```

   - **Install Python Dependencies**: Use the pre-downloaded Python packages to set up your environment:

     ```bash
     pip install --no-index --find-links=python_packages -r requirements.txt
     ```

   - **Set Up Models**: Ensure that the downloaded LLaMA 2 model is placed in the appropriate directory as expected by your Jupyter Notebook.

3. **Running the Jupyter Notebook**:

   - **Start the Ollama Server**: Run the Ollama server using Docker:

     ```bash
     docker run -d --name ollama -p 11434:11434 ollama/ollama:latest
     ```

   - **Launch Jupyter Notebook**: Navigate to your project directory and start Jupyter Notebook:

     ```bash
     jupyter notebook
     ```

   - **Access the Notebook**: Open your web browser and navigate to the Jupyter Notebook interface to open and run your notebook.

By following these steps, you can set up and run your Jupyter Notebook in an environment without internet access. 